# Notebook 1: Data Preprocessing and Cleaning
The codes in this notebook will contain all the steps done to preprocess and clean the data. The dataset used comes the [Family Income and Expenditure Survey](https://psada.psa.gov.ph/catalog/FIES/about) held by the PSA (specifically, the 2021 data). Through this project, we hope to examine the households and see the factors that determine whether a family is considered to be below or above the poverty line as well as see whether the poverty threshold should be raised or not.

## Miscellaneous

In [42]:
import pandas as pd
import numpy as np
import time
from datetime import date
from tqdm import tqdm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [56]:
fies_df = pd.read_csv("data/fies_raw.csv")
fies_df

,W_REGN,W_PROV,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,...,URB,RPROV,RPSU,BWEIGHT,RFACT,RFACT_POP,PCINC,NPCINC,RPCINC,PPCINC
0,1,28,1,6.5,290000,0,290000,0,0,340000,...,2,2800,217,100.768,103.1817,106.2332,107683.080,8.0,8.0,7.0
1,1,28,2,4.0,80000,72000,152000,20500,60000,32500,...,2,2800,217,100.768,103.1817,106.2332,81202.500,7.0,7.0,6.0
2,1,28,3,1.0,0,0,0,0,30000,81000,...,2,2800,217,100.768,103.1817,106.2332,190061.000,10.0,10.0,9.0
3,1,28,4,3.0,0,0,0,8700,170000,0,...,2,2800,217,100.768,103.1817,106.2332,71235.000,7.0,7.0,5.0
4,1,28,5,4.0,12000,80400,92400,250,30000,0,...,2,2800,217,100.768,103.1817,106.2332,68072.500,6.0,6.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165024,17,59,165025,1.0,0,0,0,0,26000,4200,...,2,5900,105,51.150,50.5465,49.4861,46524.000,4.0,5.0,7.0
165025,17,59,165026,5.0,0,0,0,0,0,0,...,2,5900,105,51.150,50.5465,49.4861,24656.000,1.0,2.0,3.0
165026,17,59,165027,3.0,55050,54900,109950,0,0,15500,...,2,5900,105,51.150,50.5465,49.4861,47986.668,4.0,6.0,7.0
165027,17,59,165028,8.0,0,3600,3600,0,5000,37800,...,2,5900,105,51.150,50.5465,49.4861,16493.750,1.0,1.0,1.0


In [27]:
fies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165029 entries, 0 to 165028
Data columns (total 88 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   W_REGN                        165029 non-null  int64  
 1   W_PROV                        165029 non-null  int64  
 2   SEQUENCE_NO                   165029 non-null  int64  
 3   FSIZE                         165029 non-null  float64
 4   REG_SAL                       165029 non-null  int64  
 5   SEASON_SAL                    165029 non-null  int64  
 6   WAGES                         165029 non-null  int64  
 7   NETSHARE                      165029 non-null  int64  
 8   CASH_ABROAD                   165029 non-null  int64  
 9   CASH_DOMESTIC                 165029 non-null  int64  
 10  RENTALS_REC                   165029 non-null  int64  
 11  INTEREST                      165029 non-null  int64  
 12  PENSION                       165029 non-nul

In [28]:
fies_df.columns

Index(['W_REGN', 'W_PROV', 'SEQUENCE_NO', 'FSIZE', 'REG_SAL', 'SEASON_SAL',
       'WAGES', 'NETSHARE', 'CASH_ABROAD', 'CASH_DOMESTIC', 'RENTALS_REC',
       'INTEREST', 'PENSION', 'DIVIDENDS', 'OTHER_SOURCE', 'NET_RECEIPT',
       'REGFT', 'NET_CFG', 'NET_LPR', 'NET_FISH', 'NET_FOR', 'NET_RET',
       'NET_MFG', 'NET_TRANS', 'NET_NEC_A8', 'NET_NEC_A9', 'NET_NEC_A10',
       'EAINC', 'TOINC', 'LOSSES', 'BREAD', 'MEAT', 'FISH', 'MILK', 'OIL',
       'FRUIT', 'VEG', 'SUGAR', 'FOOD_NEC', 'FRUIT_VEG', 'COFFEE', 'TEA',
       'COCOA', 'WATER', 'SOFTDRINKS', 'OTHER_NON_ALCOHOL', 'ALCOHOL',
       'TOBACCO', 'OTHER_VEG', 'SERVICES_PRIMARY_GOODS',
       'ALCOHOL_PROCDUCTION_SERVICES', 'FOOD_HOME', 'FOOD_OUTSIDE', 'FOOD',
       'CLOTH', 'FURNISHING', 'HEALTH', 'HOUSING_WATER', 'ACTRENT', 'RENTVAL',
       'IMPUTED_RENT', 'BIMPUTED_RENT', 'TRANSPORT', 'COMMUNICATION',
       'RECREATION', 'EDUCATION', 'INSURANCE', 'MISCELLANEOUS', 'DURABLE',
       'OCCASION', 'OTHER_EXPENDITURE', 'OTHER_DISBU

## Getting Poverty Threshold

To set whether a household is classified to be **above the poverty line** or **below the poverty line**, the 2021 poverty threshold set by the PSA was used. According to the [Official Poverty Statistics](https://www.psa.gov.ph/system/files/phdsd/2023%201st%20Sem%20Official%20Poverty%20Statistics%20Report.pdf), the monthly poverty threshold for a family of five is 12,082.

In [57]:
monthly_threshold = 12082 / 5
monthly_threshold

2416.4

In [58]:
# Is the monthly income of the household less than the monthly threshold * family size?
fies_df['IS_POVERTY'] = (fies_df['TOINC'] / 12 < fies_df['FSIZE'] * monthly_threshold).astype(int)
display(fies_df['IS_POVERTY'].value_counts(1))

IS_POVERTY
0    0.843621
1    0.156379
Name: proportion, dtype: float64

## Enconding Region and Province

Notice that the Region and Province columns of the dataset are numbers, which are codes set by PSA. We will convert this to its proper names so that the model can process this.

In [59]:
fies_df.head()[["W_REGN", "RPROV"]]

,W_REGN,RPROV
0,1,2800
1,1,2800
2,1,2800
3,1,2800
4,1,2800


### Region and Province Values

In [32]:
region = pd.DataFrame([
  {
    "region": "National Capital Region",
    "code": 13
  },
  {
    "region": "Cordillera Administrative Region",
    "code": 14
  },
  {
    "region": "Region I - Ilocos Region",
    "code": 1
  },
  {
    "region": "Region II - Cagayan Valley",
    "code": 2
  },
  {
    "region": "Region III - Central Luzon",
    "code": 3
  },
  {
    "region": "Region IVA - CALABARZON",
    "code": 4
  },
  {
    "region": "Region V- Bicol",
    "code": 5
  },
  {
    "region": "Region VI - Western Visayas",
    "code": 6
  },
  {
    "region": "Region VII - Central Visayas",
    "code": 7
  },
  {
    "region": "Region VIII - Eastern Visayas",
    "code": 8
  },
  {
    "region": "Region IX - Zamboanga Peninsula",
    "code": 9
  },
  {
    "region": "Region X - Northern Mindanao",
    "code": 10
  },
  {
    "region": "Region XI - Davao",
    "code": 11
  },
  {
    "region": "Region XII - SOCCSKSARGEN",
    "code": 12
  },
  {
    "region": "Region XIII - Caraga",
    "code": 16
  },
  {
    "region": "Autonomous Region in Muslim Mindanao",
    "code": 15
  },
  {
    "region": "Region IVB - MIMAROPA",
    "code": 17
  }
])

province = pd.DataFrame([
  {
    "province": " City of Manila",
    "code": 3900
  },
  {
    "province": " City of Mandaluyong",
    "code": 7401
  },
  {
    "province": " City of Marikina",
    "code": 7402
  },
  {
    "province": " City of Pasig",
    "code": 7403
  },
  {
    "province": " Quezon City",
    "code": 7404
  },
  {
    "province": " City of San Juan",
    "code": 7405
  },
  {
    "province": " Caloocan City",
    "code": 7501
  },
  {
    "province": " City of Malabon",
    "code": 7502
  },
  {
    "province": " City of Navotas",
    "code": 7503
  },
  {
    "province": " City of Valenzuela",
    "code": 7504
  },
  {
    "province": " City of Las Piñas",
    "code": 7601
  },
  {
    "province": " City of Makati",
    "code": 7602
  },
  {
    "province": " City of Muntinlupa",
    "code": 7603
  },
  {
    "province": " City of Parañaque",
    "code": 7604
  },
  {
    "province": " Pasay City",
    "code": 7605
  },
  {
    "province": " Pateros",
    "code": 7600
  },
  {
    "province": " Taguig City",
    "code": 7607
  },
  {
    "province": "Ilocos Norte",
    "code": 2800
  },
  {
    "province": "Ilocos Sur",
    "code": 2900
  },
  {
    "province": "La Union",
    "code": 3300
  },
  {
    "province": "Pangasinan",
    "code": 5500
  },
  {
    "province": "Batanes",
    "code": 900
  },
  {
    "province": "Cagayan",
    "code": 1500
  },
  {
    "province": "Isabela",
    "code": 3100
  },
  {
    "province": "Nueva Vizcaya",
    "code": 5000
  },
  {
    "province": "Quirino",
    "code": 5700
  },
  {
    "province": "Aurora",
    "code": 7700
  },
  {
    "province": "Bataan",
    "code": 800
  },
  {
    "province": "Bulacan",
    "code": 1400
  },
  {
    "province": "Nueva Ecija",
    "code": 4900
  },
  {
    "province": " Pampanga(Excluding Angeles City)",
    "code": 5400
  },
  {
    "province": " Angeles City",
    "code": 5401
  },
  {
    "province": "Tarlac",
    "code": 6900
  },
  {
    "province": " Zambales (Excluding Olongapo City)",
    "code": 7100
  },
  {
    "province": " Olongapo City",
    "code": 7107
  },
  {
    "province": "Batangas",
    "code": 1000
  },
  {
    "province": "Cavite",
    "code": 2100
  },
  {
    "province": "Laguna",
    "code": 3400
  },
  {
    "province": " Quezon (Excluding Lucena City)",
    "code": 5600
  },
  {
    "province": " Lucena City",
    "code": 5624
  },
  {
    "province": "Rizal",
    "code": 5800
  },
  {
    "province": "Marinduque",
    "code": 4000
  },
  {
    "province": "Occ Mindoro",
    "code": 5100
  },
  {
    "province": "Or Mindoro",
    "code": 5200
  },
  {
    "province": " Palawan (Excluding Puerto Princesa City)",
    "code": 5300
  },
  {
    "province": " Puerto Princesa City",
    "code": 5316
  },
  {
    "province": "Romblon",
    "code": 5900
  },
  {
    "province": "Albay",
    "code": 500
  },
  {
    "province": "Cam Norte",
    "code": 1600
  },
  {
    "province": "Cam Sur",
    "code": 1700
  },
  {
    "province": "Catanduanes",
    "code": 2000
  },
  {
    "province": "Masbate",
    "code": 4100
  },
  {
    "province": "Sorsogon",
    "code": 6200
  },
  {
    "province": "Aklan",
    "code": 400
  },
  {
    "province": "Antique",
    "code": 600
  },
  {
    "province": "Capiz",
    "code": 1900
  },
  {
    "province": "Guimaras",
    "code": 7900
  },
  {
    "province": " Iloilo (Excluding Iloilo City)",
    "code": 3000
  },
  {
    "province": " Iloilo City",
    "code": 3022
  },
  {
    "province": "Negros Occidental (Excluding Bacolod City)",
    "code": 4500
  },
  {
    "province": "Bacolod City",
    "code": 4501
  },
  {
    "province": "Bohol",
    "code": 1200
  },
  {
    "province": "Other Cebu (Excluding Cebu City, Lapu-Lapu City and Mandaue City)",
    "code": 2200
  },
  {
    "province": "Cebu City",
    "code": 2217
  },
  {
    "province": "Lapu-Lapu City",
    "code": 2226
  },
  {
    "province": "Mandaue City",
    "code": 2230
  },
  {
    "province": "Negros Oriental",
    "code": 4600
  },
  {
    "province": "Siquijor",
    "code": 6100
  },
  {
    "province": "Biliran",
    "code": 7800
  },
  {
    "province": "E. Samar",
    "code": 2600
  },
  {
    "province": "Leyte (Excluding Tacloban City)",
    "code": 3700
  },
  {
    "province": " Tacloban City",
    "code": 3747
  },
  {
    "province": "N. Samar",
    "code": 4800
  },
  {
    "province": "W. Samar",
    "code": 6000
  },
  {
    "province": "S. Leyte",
    "code": 6400
  },
  {
    "province": "Zambo Norte",
    "code": 7200
  },
  {
    "province": "Zamboanga del Sur (Excluding Zamboanga City)",
    "code": 7300
  },
  {
    "province": " Zamboanga City",
    "code": 7332
  },
  {
    "province": "Zambo Sibugay",
    "code": 8300
  },
  {
    "province": "City of Isabela",
    "code": 9700
  },
  {
    "province": "Bukidnon",
    "code": 1300
  },
  {
    "province": "Camiguin",
    "code": 1800
  },
  {
    "province": "Lanao del Norte (Excluding Iligan City)",
    "code": 3500
  },
  {
    "province": "Iligan City",
    "code": 3504
  },
  {
    "province": "Misamis Occ",
    "code": 4200
  },
  {
    "province": "Misamis Oriental (Excluding Cagayan de Oro City)",
    "code": 4300
  },
  {
    "province": "Cagayan de Oro City",
    "code": 4305
  },
  {
    "province": "Compostela V",
    "code": 8200
  },
  {
    "province": "Davao Norte",
    "code": 2300
  },
  {
    "province": "Davao del Sur (Excluding Davao City and Davao Occidental)",
    "code": 2400
  },
  {
    "province": "Davao City",
    "code": 2402
  },
  {
    "province": "Davao Oriental",
    "code": 2500
  },
  {
    "province": "Davao Occidental",
    "code": 8600
  },
  {
    "province": "N. Cotabato",
    "code": 4700
  },
  {
    "province": "Sarangani",
    "code": 8000
  },
  {
    "province": "South Cotabato (Excluding General Santos City)",
    "code": 6300
  },
  {
    "province": "General Santos City",
    "code": 6303
  },
  {
    "province": "Sultan Kudarat",
    "code": 6500
  },
  {
    "province": "Cotabato City",
    "code": 9804
  },
  {
    "province": "Abra",
    "code": 100
  },
  {
    "province": "Apayao",
    "code": 8100
  },
  {
    "province": "Benguet (Excluding Baguio City)",
    "code": 1100
  },
  {
    "province": "Baguio City",
    "code": 1102
  },
  {
    "province": "Ifugao",
    "code": 2700
  },
  {
    "province": "Kalinga",
    "code": 3200
  },
  {
    "province": "Mt Province",
    "code": 4400
  },
  {
    "province": "Basilan",
    "code": 700
  },
  {
    "province": "Lanao del Sur",
    "code": 3600
  },
  {
    "province": "Maguindanao",
    "code": 3800
  },
  {
    "province": "Sulu",
    "code": 6600
  },
  {
    "province": "Tawi-Tawi",
    "code": 7000
  },
  {
    "province": "Agusan del Norte (Excluding Butuan City)",
    "code": 200
  },
  {
    "province": "Butuan City",
    "code": 202
  },
  {
    "province": "Agusan Sur",
    "code": 300
  },
  {
    "province": "Surigao Norte",
    "code": 6700
  },
  {
    "province": "Surigao Sur",
    "code": 6800
  },
  {
    "province": "Dinagat Islands",
    "code": 8500
  }
])

### Replacing Region and Province

In [60]:
# Region
fies_df = fies_df.merge(region, how='left', left_on='W_REGN', right_on='code').drop("code", axis=1)

# Province
fies_df = fies_df.merge(province, how='left', left_on='RPROV', right_on='code').drop("code", axis=1)

# Dropping columns
fies_df = fies_df.drop(columns=["W_REGN", "W_PROV", "RPROV"], axis=1)

fies_df.head()

,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,RENTALS_REC,INTEREST,...,BWEIGHT,RFACT,RFACT_POP,PCINC,NPCINC,RPCINC,PPCINC,IS_POVERTY,region,province
0,1,6.5,290000,0,290000,0,0,340000,0,0,...,100.768,103.1817,106.2332,107683.08,8.0,8.0,7.0,0,Region I - Ilocos Region,Ilocos Norte
1,2,4.0,80000,72000,152000,20500,60000,32500,0,0,...,100.768,103.1817,106.2332,81202.50,7.0,7.0,6.0,0,Region I - Ilocos Region,Ilocos Norte
2,3,1.0,0,0,0,0,30000,81000,0,0,...,100.768,103.1817,106.2332,190061.00,10.0,10.0,9.0,0,Region I - Ilocos Region,Ilocos Norte
3,4,3.0,0,0,0,8700,170000,0,0,0,...,100.768,103.1817,106.2332,71235.00,7.0,7.0,5.0,0,Region I - Ilocos Region,Ilocos Norte
4,5,4.0,12000,80400,92400,250,30000,0,0,0,...,100.768,103.1817,106.2332,68072.50,6.0,6.0,4.0,0,Region I - Ilocos Region,Ilocos Norte


### Initial Saving of Dataset

In [61]:
# fies_df.to_csv("data/cleaned_fies.csv", index=False)

### One-Hot Encoding

In [62]:
to_drop = []
for col in tqdm(fies_df.columns):
    if fies_df[col].dtype == "O":
        dummies = pd.get_dummies(fies_df[col], prefix=col, drop_first=False)
        fies_df = pd.concat([fies_df, dummies], axis=1)
        to_drop.append(col)

fies_df.drop(columns=col, axis=1, inplace=True)
fies_df.drop(columns=['region'], axis=1, inplace=True)
fies_df

100%|██████████| 88/88 [00:00<00:00, 345.88it/s]


,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,RENTALS_REC,INTEREST,...,province_Sultan Kudarat,province_Sulu,province_Surigao Norte,province_Surigao Sur,province_Tarlac,province_Tawi-Tawi,province_W. Samar,province_Zambo Norte,province_Zambo Sibugay,province_Zamboanga del Sur (Excluding Zamboanga City)
0,1,6.5,290000,0,290000,0,0,340000,0,0,...,False,False,False,False,False,False,False,False,False,False
1,2,4.0,80000,72000,152000,20500,60000,32500,0,0,...,False,False,False,False,False,False,False,False,False,False
2,3,1.0,0,0,0,0,30000,81000,0,0,...,False,False,False,False,False,False,False,False,False,False
3,4,3.0,0,0,0,8700,170000,0,0,0,...,False,False,False,False,False,False,False,False,False,False
4,5,4.0,12000,80400,92400,250,30000,0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165024,165025,1.0,0,0,0,0,26000,4200,0,0,...,False,False,False,False,False,False,False,False,False,False
165025,165026,5.0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
165026,165027,3.0,55050,54900,109950,0,0,15500,0,0,...,False,False,False,False,False,False,False,False,False,False
165027,165028,8.0,0,3600,3600,0,5000,37800,0,0,...,False,False,False,False,False,False,False,False,False,False


### Type Conversion

In [63]:
for col in fies_df.columns:
    if fies_df[col].dtype == "bool":
        fies_df[col] = fies_df[col].astype(int)

fies_df.head()

,SEQUENCE_NO,FSIZE,REG_SAL,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,RENTALS_REC,INTEREST,...,province_Sultan Kudarat,province_Sulu,province_Surigao Norte,province_Surigao Sur,province_Tarlac,province_Tawi-Tawi,province_W. Samar,province_Zambo Norte,province_Zambo Sibugay,province_Zamboanga del Sur (Excluding Zamboanga City)
0,1,6.5,290000,0,290000,0,0,340000,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,4.0,80000,72000,152000,20500,60000,32500,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1.0,0,0,0,0,30000,81000,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,3.0,0,0,0,8700,170000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,4.0,12000,80400,92400,250,30000,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Saving Dataset

In [64]:
# fies_df.to_csv("data/one_hot_encoded_fies.csv", index=False)

## Variance Inflation Factor (VIF)

VIF was done to reduce the number of columns (or features) the dataset has. This was done to make it more efficient for the model. Simply, we use VIF iteratively and remove the columns/features that are correlated to each other.

In [74]:
to_drop = ["SEQUENCE_NO", "RPSU", "TOINC", "BWEIGHT", "RFACT", "RFACT_POP", "PCINC", "NPCINC", "PPCINC", "RPCINC", "IS_POVERTY",
            'region_Region I - Ilocos Region']

vif_df = fies_df.drop(columns=to_drop, axis=1)

# remove columns with "provinces"
vif_df = vif_df.drop(columns=[col for col in vif_df.columns if "province_" in col], axis=1)

# Calculate initial VIF values
dropped_cols = []
vif_data = pd.DataFrame()
vif_data['Feature'] = vif_df.columns
vif_data['VIF'] = [variance_inflation_factor(vif_df.values, i) for i in range(len(vif_df.columns))]

count = 0

# Iterate until the maximum VIF <= 5
while vif_data["VIF"].max() > 5:
    idx = vif_data['VIF'].idxmax() # get index with max VIF
    feat = vif_data.loc[idx, 'Feature'] # get feature with max VIF
    dropped_cols.append(feat)
    vif_df = vif_df.drop(columns=feat, axis=1) # drop the column

    # Printing
    print(f"Iteration {count}")
    print(f"{vif_data.sort_values(by='VIF', ascending=False)}\n")

    # Recalculate VIF
    vif_data = pd.DataFrame()
    vif_data['Feature'] = vif_df.columns
    vif_data['VIF'] = [variance_inflation_factor(vif_df.values, i) for i in range(len(vif_df.columns))]

    count += 1

print("Final VIF")
display(vif_data)

print("\nDropped Columns")
print(dropped_cols)

Iteration 0
                                   Feature           VIF
1                                  REG_SAL           inf
2                               SEASON_SAL           inf
3                                    WAGES           inf
67                      OTHER_DISBURSEMENT  1.510261e+12
54                                 ACTRENT  1.173118e+12
..                                     ...           ...
88                region_Region XI - Davao  1.553635e+00
90             region_Region XIII - Caraga  1.532378e+00
89        region_Region XII - SOCCSKSARGEN  1.502241e+00
82  region_Region IX - Zamboanga Peninsula  1.475879e+00
41                       OTHER_NON_ALCOHOL  1.029358e+00

[91 rows x 2 columns]

Iteration 1
                                   Feature           VIF
69                                   TOTEX  9.007199e+15
68                                   NFOOD  3.002400e+15
48                                    FOOD  1.000800e+15
70                                  TOTD

,Feature,VIF
0,SEASON_SAL,1.336808
1,NETSHARE,1.051974
2,CASH_ABROAD,1.253284
3,CASH_DOMESTIC,1.358257
4,RENTALS_REC,1.195861
...,...,...
71,region_Region VIII - Eastern Visayas,1.234615
72,region_Region X - Northern Mindanao,1.144833
73,region_Region XI - Davao,1.130598
74,region_Region XII - SOCCSKSARGEN,1.144082



Dropped Columns
['REG_SAL', 'TOTEX', 'TOTDIS', 'RENTVAL', 'EAINC', 'FOOD', 'TOREC', 'NFOOD', 'FOOD_HOME', 'HOUSING_WATER', 'URB', 'BREAD', 'FSIZE', 'WAGES', 'FISH']


### Dropping the Columns

In [86]:
# reinclude the ff. columns: sequence_no, 'BREAD', 'WAGES', 'FISH', IS_POVERTY

df = pd.read_csv('data/one_hot_encoded_fies.csv')
columns_to_drop = ['region_Region I - Ilocos Region','REG_SAL', 'TOTEX', 'TOTDIS', 'RENTVAL', 'EAINC', 'FOOD', 'TOREC', 'NFOOD', 'FOOD_HOME', 'HOUSING_WATER', 'URB', 'FSIZE', "RPSU", "TOINC", "BWEIGHT", "RFACT", "RFACT_POP", "PCINC", "NPCINC", "PPCINC", "RPCINC",]

final_df = df.copy()
final_df.drop(columns=columns_to_drop, axis=1, inplace=True)
final_df.drop(columns=[col for col in final_df.columns if "province_" in col], axis=1, inplace=True)
final_df.head()

,SEQUENCE_NO,SEASON_SAL,WAGES,NETSHARE,CASH_ABROAD,CASH_DOMESTIC,RENTALS_REC,INTEREST,PENSION,DIVIDENDS,...,region_Region IVB - MIMAROPA,region_Region IX - Zamboanga Peninsula,region_Region V- Bicol,region_Region VI - Western Visayas,region_Region VII - Central Visayas,region_Region VIII - Eastern Visayas,region_Region X - Northern Mindanao,region_Region XI - Davao,region_Region XII - SOCCSKSARGEN,region_Region XIII - Caraga
0,1,0,290000,0,0,340000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,72000,152000,20500,60000,32500,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,30000,81000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,8700,170000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,80400,92400,250,30000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Saving the final dataset

In [90]:
final_df.to_csv("data/final_fies_data.csv", index=False)